# Trying out `EvolvingGraphs`

We're interested in using the [`EvolvingGraphs`](http://evolvinggraphsjl.readthedocs.io/en/latest/overview.html).

In [4]:
Pkg.add("EvolvingGraphs")
using EvolvingGraphs

INFO: Nothing to be done


The [tutorial for the EvolvingGraphs](http://evolvinggraphsjl.readthedocs.io/en/latest/tutorial.html) package has some stuff we can work through.

Let's start by creating a graph with a couple of edges at different times:

In [17]:
graph = evolving_graph()

Directed EvolvingGraph (0 nodes, 0 static edges, 0 timestamps)

In [18]:
add_edge!(graph, 1, 2, 1)

Node(1)->Node(2) at time 1

In [19]:
timestamps(graph)

1-element Array{Int64,1}:
 1

In [21]:
add_edge!(graph, 1, 3, 2)
timestamps(graph)

2-element Array{Int64,1}:
 1
 2

In [22]:
edges(graph)

2-element Array{EvolvingGraphs.TimeEdge{EvolvingGraphs.Node{Int64},Int64},1}:
 Node(1)->Node(2) at time 1
 Node(1)->Node(3) at time 2

This graph is _directed_, which is perfect for our use case. We should be able to reconstruct simple SKOS hierarchies in this system. Let's try playing with weighted edges.